# Traveling Salesman Problem
The input data for the traveling salesman problem is given in the following format:
- the first line contains a single number, the number of cities, $n$, that the salesman must travel to
- the remaining $n$ lines contain the $(x,y)$ coordinates of each of the $n$ cities

The file name itself indicates how many cities there are. For example: "tsp_100_1" and "tsp_100_2" are two unique datasets containing 100 cities.

In [2]:
import math
from collections import namedtuple

import numpy as np
import networkx as nx

My initial attempt to solve the problem involved using NetworkX and the Hungarian Algorithm. Essentially, I ran the distance matrix through the Hungarian algorithm. This left me with a matrix containing at least one zero in each row and column. I used a function defined below, called get_graph, in order to create a new graph and adjacency matrix, where the $ij^{th}$ entry of this new matrix was 1 if the $ij^{th}$ entry of the original matrix was 0. If I could find a Hamiltonian cycle for this new graph, then I would be guaranteed to have a route with minimum distance. If such a cycle didn't exist, I would subtract the minimum value of the original matrix (excluding zero) from all the other non-zero values, and repeated the process. What this essentially did was slowly add edges to the matrix from which I was trying to find a Hamiltonian cycle. The idea would be to keep doing this until I could find a Hamiltonian cycle. The problem that I ran into was that it is increibly computationally expensive to look for Hamiltonian cycles, and the more edges I add, the longer it takes. Admittedly, I am sure there are quicker ways to find Hamiltonian cycles than what I was doing (although it is an NP hard problem). Even though it would give me an optimal solution, I ultimately decided to pursue different ways of solving the problem. However, some of my original code is given below:

In [14]:
def get_hungarian(matrix):
    for i in range(nodeCount):
        matrix[i][:] -= min(matrix[i][:])
        
    for j in range(nodeCount):
        matrix[:][j] -= min(matrix[:][j])   
    return matrix    

def subtract_min_val(matrix):
    minval = np.min(matrix[np.nonzero(matrix)])
    matrix[np.nonzero(matrix)] -= minval  
    return matrix

def get_graph(matrix):
    G=nx.Graph()
    node_list = []
    for i in range(nodeCount):
        for j in range(nodeCount):
            if hung_matrix[i][j] == 0:
                G.add_edges_from([(i,j)])
                node_list.append(i)
                node_list.append(j)
                
    A = nx.adjacency_matrix(G, nodelist = list(set(node_list)))

    #draw graph:
    nx.draw(G, with_labels=True,node_color = 'black', node_size = 40)

    #uncomment to print adjacency matrix:
    #print('Adjacency Matrix: \n', A.todense())
    
    return np.array(A.todense())

This is an example of how it was implemented below. Note: I used a Hamiltonian solver I had found online called hamCycle that also required me to define a class Graph. 


In [ ]:
Point = namedtuple("Point", ['x', 'y'])

f = open('data/tsp_5_1', 'r')
input_data = f.read()

lines = input_data.split('\n')

nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    points.append(Point(float(parts[0]), float(parts[1])))


distance_matrix = np.zeros((nodeCount, nodeCount))
for i in range(nodeCount):
    for j in range(nodeCount):
        distance_matrix[i][j] = length(points[i], points[j])
for k in range(nodeCount):
    distance_matrix[k][k] = 1000000000000000

distance_matrix_copy = distance_matrix.copy()    
    
hung_matrix = get_hungarian(distance_matrix_copy)

adj_matrix = get_graph(hung_matrix)


g = Graph(nodeCount)  
g.graph = adj_matrix


while g.hamCycle() == False:
    hung_matrix = subtract_min_val(hung_matrix)
    adj_matrix = get_graph(hung_matrix)
    
    g = Graph(nodeCount)  
    g.graph = adj_matrix

    
solution = g.hamCycle()

total_distance = 0
total_distance += distance_matrix[solution[-1]][solution[0]]
for d in range(nodeCount - 1):
    total_distance += distance_matrix[solution[d]][solution[d+1]]

print(solution)
print('Distance: ', total_distance)

### Greedy Nearest Neighbor
Next, I tried a greedy nearest neighbor approach. I would start at an arbitrary node $x$, and move to the closest node (using the distance matrix to find the smallest distance). I would continue in this way, always moving to the closest unused node, until all of the nodes had been traveled to. Since this method is not too computationaly computationally expensive, as long as there arent too many cities, I can run this algorithm beginning at every node, and choose the shortest distance out of them all. 

In [6]:

from tqdm import tqdm

#greedy algorithm to find upper bound for distance

Point = namedtuple("Point", ['x', 'y'])

def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

f = open('data/tsp_574_1', 'r')
input_data = f.read()

lines = input_data.split('\n')

nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    points.append(Point(float(parts[0]), float(parts[1])))


distance_matrix = np.zeros((nodeCount, nodeCount))
for i in range(nodeCount):
    for j in range(nodeCount):
        distance_matrix[i][j] = length(points[i], points[j])
for k in range(nodeCount):
    distance_matrix[k][k] = 1000000000

    
    
#create a greedy route starting with node x

def nearest_neighbor_greedy(x):
    greedy_route = [x]
    greedy_distance = 0
    nodes_left = list(range(nodeCount))
    nodes_left.remove(x)
    
    for i in range(nodeCount - 2):
        greedy_distance += min(distance_matrix[greedy_route[i]][nodes_left])
        greedy_route.append(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
        nodes_left.remove(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
    greedy_distance += distance_matrix[greedy_route[-1]][nodes_left[0]] + distance_matrix[x][nodes_left[0]]

    greedy_route.append(nodes_left[0])
    return greedy_distance, greedy_route
    
current_min = 100000000000
best_route = []

for j in tqdm(range(nodeCount)):
    distance, route = nearest_neighbor_greedy(j)
    if distance < current_min:
        current_min = distance
        best_route = route

#print(best_route)
print(current_min)
    

100%|██████████| 574/574 [00:27<00:00, 20.70it/s]

44514.83535435058


### Greedy Nearest Neighbor with local greedy 2-swap

Use the greedy algorithm above to get a decent route. Then, search through every two nodes within that route, and if swapping the nodes improves the route, swap them. Otherwise, leave them unchanged

In [9]:

from tqdm import tqdm


#greedy algorithm to find upper bound for distance

Point = namedtuple("Point", ['x', 'y'])

f = open('data/tsp_574_1', 'r')
input_data = f.read()

lines = input_data.split('\n')

nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    points.append(Point(float(parts[0]), float(parts[1])))


distance_matrix = np.zeros((nodeCount, nodeCount))
for i in range(nodeCount):
    for j in range(nodeCount):
        distance_matrix[i][j] = length(points[i], points[j])
for k in range(nodeCount):
    distance_matrix[k][k] = 1000000000


    
def get_current_cost(route):
    current_cost = 0
    for i in range(len(route) - 1):
        current_cost += distance_matrix[route[i]][route[i+1]]
    current_cost += distance_matrix[route[0]][route[-1]]
    return current_cost
    
#create a greedy route starting with node x

def nearest_neighbor_greedy(x):
    greedy_route = [x]
    greedy_distance = 0
    nodes_left = list(range(nodeCount))
    nodes_left.remove(x)
    
    for i in range(nodeCount - 2):
        greedy_distance += min(distance_matrix[greedy_route[i]][nodes_left])
        greedy_route.append(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
        nodes_left.remove(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
    greedy_distance += distance_matrix[greedy_route[-1]][nodes_left[0]] + distance_matrix[x][nodes_left[0]]

    greedy_route.append(nodes_left[0])
    return greedy_distance, greedy_route
    
current_min = 100000000000
best_route = []


for j in range(nodeCount):
    distance, route = nearest_neighbor_greedy(j)
    if distance < current_min:
        current_min = distance
        best_route = route.copy()
        
        
def greedy_two_swap(route):
    for i in range(nodeCount):
        for j in range(nodeCount):
            if i != j: 
                test_1 = route.copy()
                test_1[i], test_1[j] = test_1[j], test_1[i]
                if get_current_cost(test_1) < get_current_cost(route):
                    route = test_1.copy()
                    current_min = get_current_cost(route)

    return route

for i in tqdm(range(10)):
    best_route = greedy_two_swap(best_route)
    current_min = get_current_cost(best_route)


current_min = get_current_cost(best_route)

#print(best_route)
print(current_min)
    

100%|██████████| 10/10 [25:16<00:00, 151.70s/it]

43606.73743948595


### Greedy Nearest Neighbor with local 2-opt search

Use the greedy algorithm above to get a decent route. Then, perform the 2-opt algorithm: 
- delete two edges from four distinct nodes
- connect the four nodes with two edges (different from the deleted edges) in a way that assures the route is still a cycle
- check to see if the new route has a shorter distance than the original route

This algorithm is a great way to remove any crossings in your route (an optimal route will never contain any intersecting edges). Below, I perform 10 iterations of the 2-opt algorithm. By inspection, it seems that after a few iterations, a local minimum is achieved, and the algorithm does not improve the score further.

In [11]:

from tqdm import tqdm



def tsp(filename):

    #greedy algorithm to find upper bound for distance

    Point = namedtuple("Point", ['x', 'y'])
    
    def length(point1, point2):
        return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
    
    #f = open('data/tsp_574_1', 'r')
    f = open(filename, 'r')
    input_data = f.read()

    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))


    distance_matrix = np.zeros((nodeCount, nodeCount))
    for i in range(nodeCount):
        for j in range(nodeCount):
            distance_matrix[i][j] = length(points[i], points[j])
    for k in range(nodeCount):
        distance_matrix[k][k] = 1000000000



    def get_current_cost(route):
        current_cost = 0
        for i in range(len(route) - 1):
            current_cost += distance_matrix[route[i]][route[i+1]]
        current_cost += distance_matrix[route[0]][route[-1]]
        return current_cost

    #create a greedy route starting with node x

    def nearest_neighbor_greedy(x):
        greedy_route = [x]
        greedy_distance = 0
        nodes_left = list(range(nodeCount))
        nodes_left.remove(x)

        for i in range(nodeCount - 2):
            greedy_distance += min(distance_matrix[greedy_route[i]][nodes_left])
            greedy_route.append(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
            nodes_left.remove(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
        greedy_distance += distance_matrix[greedy_route[-1]][nodes_left[0]] + distance_matrix[x][nodes_left[0]]

        greedy_route.append(nodes_left[0])
        return greedy_distance, greedy_route

    current_min = 100000000000
    best_route = []


    for j in range(nodeCount):
        distance, route = nearest_neighbor_greedy(j)
        if distance < current_min:
            current_min = distance
            best_route = route.copy()


    def two_opt(route, i, j):
        first_chunk = route[0:i]
        second_chunk = route[i:j+1]
        second_chunk.reverse()
        third_chunk = route[j+1:]

        test_route = first_chunk + second_chunk + third_chunk
        test_distance = get_current_cost(test_route)

        return test_route, test_distance



    print('before 2-opt:', current_min)




    for o in range(10):
        for i in range(nodeCount):
            for j in range(nodeCount):
                if i <= j:
                    test_route, test_distance = two_opt(best_route, i, j)
                    if test_distance < current_min:
                        current_min = test_distance
                        best_route = test_route.copy()
    print('after 2-opt:', current_min)



In [12]:
filename = 'data/tsp_51_1'
tsp(filename)

before 2-opt: 496.4090227267278
after 2-opt: 452.81638321759243


In [13]:
filename = 'data/tsp_100_3'
tsp(filename)

before 2-opt: 23566.4029464274
after 2-opt: 21939.46322520273


In [14]:
filename = 'data/tsp_200_2'
tsp(filename)

before 2-opt: 35394.00871886393
after 2-opt: 32000.004226562443


In [15]:
filename = 'data/tsp_574_1'
tsp(filename)

before 2-opt: 44514.83535435058
after 2-opt: 40088.548799603705


#### In order to find a solution for larger problems (and not take forever), we restrict the nearest neighbor search to only calculate a greedy route for a single node (in this case, node 0), rather than compare the greedy search for every node

In [3]:

def tsp_big(filename):

    #greedy algorithm to find upper bound for distance

    Point = namedtuple("Point", ['x', 'y'])
    
    def length(point1, point2):
        return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
    
    #f = open('data/tsp_574_1', 'r')
    f = open(filename, 'r')
    input_data = f.read()

    lines = input_data.split('\n')

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))


    distance_matrix = np.zeros((nodeCount, nodeCount))
    for i in range(nodeCount):
        for j in range(nodeCount):
            distance_matrix[i][j] = length(points[i], points[j])
    for k in range(nodeCount):
        distance_matrix[k][k] = 1000000000



    def get_current_cost(route):
        current_cost = 0
        for i in range(len(route) - 1):
            current_cost += distance_matrix[route[i]][route[i+1]]
        current_cost += distance_matrix[route[0]][route[-1]]
        return current_cost

    #create a greedy route starting with node x

    def nearest_neighbor_greedy(x):
        greedy_route = [x]
        greedy_distance = 0
        nodes_left = list(range(nodeCount))
        nodes_left.remove(x)

        for i in range(nodeCount - 2):
            greedy_distance += min(distance_matrix[greedy_route[i]][nodes_left])
            greedy_route.append(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
            nodes_left.remove(nodes_left[np.argmin(distance_matrix[greedy_route[i]][nodes_left])])
        greedy_distance += distance_matrix[greedy_route[-1]][nodes_left[0]] + distance_matrix[x][nodes_left[0]]

        greedy_route.append(nodes_left[0])
        return greedy_distance, greedy_route

    current_min = 100000000000
    best_route = []


    for j in range(1):
        distance, route = nearest_neighbor_greedy(j)
        if distance < current_min:
            current_min = distance
            best_route = route.copy()


    def two_opt(route, i, j):
        first_chunk = route[0:i]
        second_chunk = route[i:j+1]
        second_chunk.reverse()
        third_chunk = route[j+1:]

        test_route = first_chunk + second_chunk + third_chunk
        test_distance = get_current_cost(test_route)

        return test_route, test_distance



    print('before 2-opt:', current_min)




    for o in range(5):
        for i in range(nodeCount):
            for j in range(nodeCount):
                if i <= j:
                    test_route, test_distance = two_opt(best_route, i, j)
                    if test_distance < current_min:
                        current_min = test_distance
                        best_route = test_route.copy()
    print('after 2-opt:', current_min)



In [3]:
filename = 'data/tsp_1889_1'
tsp_big(filename)

before 2-opt: 391470.44549188914
after 2-opt: 348665.92917653406


In [ ]:
filename = 'data/tsp_33810_1'
tsp_big(filename)

before 2-opt: 78478867.03022146


In general, it seems that the 2-opt algorithm finds a shorter route than the greedy 2-swap. However, both algorithms quickly get stuck in a local minimum. There are methods to get around this, such as Tabu search, k-opt searches, starting with different initial routes, and/or implementing both the 2-opt and the greedy 2-swap. 

# OR-Tools

Google OR-Tools provides a great way to solve many discrete optimization problems, including the traveling salesman problem.

In [31]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def OR_tsp_solver(filename):
    
    Point = namedtuple("Point", ['x', 'y'])

    def length(point1, point2):
        return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

    #f = open('data/tsp_574_1', 'r')
    f = open(filename, 'r')
    input_data = f.read()

    lines = input_data.split('\n')

    nodeCount = int(lines[0])
    
    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))


    distance_matrix = np.zeros((nodeCount, nodeCount))
    for i in range(nodeCount):
        for j in range(nodeCount):
            distance_matrix[i][j] = length(points[i], points[j])
  
    
    def create_data_model():
        data = {}
        data['distance_matrix'] = distance_matrix
        
        #OR-tools allows for multiple vehicles, which is not necessary for our current problem
        data['num_vehicles'] = 1
        
        #which node to start the route (and end) the route
        data['depot'] = 0
        return data
    
    
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        #function which uses the distance matrix to retrieve the distance between any two nodes
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
    #the "cost" is determined by distance between two locations
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    #an initial route is found using the nearest neighbors strategy, as was done in my previous solutions
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    def print_solution(manager, routing, solution):
        #Prints solution 
        print('Objective: {} miles'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        plan_output = 'Route for vehicle 0:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        print(plan_output)
        plan_output += 'Route distance: {}miles\n'.format(route_distance)
    
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution)

#### Unsurprisingly, OR-tools does a better job minimizing the objective, and is more effecient.

However, the code above would need to be tweaked for the case of 33810 cities (I tried running the script for several hours before giving up).

In [33]:
filename = 'data/tsp_51_1'
OR_tsp_solver(filename)

Objective: 424 miles
Route for vehicle 0:
 0 -> 5 -> 2 -> 28 -> 10 -> 9 -> 45 -> 27 -> 47 -> 26 -> 6 -> 36 -> 12 -> 30 -> 23 -> 34 -> 24 -> 41 -> 3 -> 46 -> 8 -> 4 -> 35 -> 13 -> 7 -> 19 -> 40 -> 18 -> 16 -> 44 -> 14 -> 15 -> 38 -> 29 -> 42 -> 11 -> 37 -> 21 -> 43 -> 50 -> 39 -> 49 -> 17 -> 32 -> 48 -> 31 -> 25 -> 20 -> 1 -> 22 -> 33 -> 0



In [32]:
filename = 'data/tsp_100_3'
OR_tsp_solver(filename)

Objective: 21499 miles
Route for vehicle 0:
 0 -> 12 -> 93 -> 97 -> 33 -> 60 -> 1 -> 36 -> 45 -> 46 -> 30 -> 94 -> 82 -> 49 -> 85 -> 6 -> 23 -> 18 -> 52 -> 22 -> 8 -> 90 -> 38 -> 79 -> 17 -> 51 -> 84 -> 72 -> 70 -> 19 -> 25 -> 40 -> 43 -> 44 -> 99 -> 11 -> 32 -> 21 -> 35 -> 54 -> 92 -> 5 -> 20 -> 87 -> 88 -> 77 -> 37 -> 47 -> 7 -> 83 -> 39 -> 74 -> 66 -> 57 -> 71 -> 24 -> 3 -> 55 -> 96 -> 80 -> 16 -> 4 -> 91 -> 69 -> 13 -> 28 -> 64 -> 34 -> 50 -> 2 -> 89 -> 76 -> 62 -> 14 -> 29 -> 26 -> 9 -> 53 -> 61 -> 95 -> 73 -> 81 -> 10 -> 75 -> 56 -> 31 -> 27 -> 58 -> 86 -> 78 -> 67 -> 98 -> 42 -> 63 -> 48 -> 59 -> 41 -> 68 -> 15 -> 65 -> 0



In [34]:
filename = 'data/tsp_200_2'
OR_tsp_solver(filename)

Objective: 30539 miles
Route for vehicle 0:
 0 -> 155 -> 199 -> 125 -> 161 -> 31 -> 104 -> 96 -> 166 -> 97 -> 169 -> 48 -> 138 -> 69 -> 152 -> 88 -> 109 -> 167 -> 10 -> 89 -> 16 -> 139 -> 93 -> 168 -> 49 -> 174 -> 129 -> 33 -> 80 -> 179 -> 119 -> 137 -> 51 -> 7 -> 65 -> 37 -> 148 -> 185 -> 22 -> 41 -> 172 -> 184 -> 21 -> 192 -> 110 -> 102 -> 57 -> 127 -> 28 -> 190 -> 196 -> 175 -> 198 -> 107 -> 128 -> 35 -> 158 -> 74 -> 66 -> 131 -> 6 -> 170 -> 60 -> 111 -> 73 -> 197 -> 194 -> 100 -> 189 -> 120 -> 45 -> 145 -> 124 -> 108 -> 126 -> 75 -> 160 -> 134 -> 71 -> 56 -> 30 -> 98 -> 44 -> 165 -> 32 -> 67 -> 13 -> 186 -> 103 -> 105 -> 182 -> 81 -> 163 -> 113 -> 24 -> 19 -> 141 -> 101 -> 8 -> 9 -> 181 -> 20 -> 46 -> 132 -> 114 -> 11 -> 27 -> 150 -> 55 -> 94 -> 147 -> 130 -> 162 -> 25 -> 86 -> 112 -> 54 -> 177 -> 116 -> 91 -> 140 -> 47 -> 1 -> 143 -> 29 -> 99 -> 191 -> 50 -> 118 -> 34 -> 61 -> 36 -> 195 -> 18 -> 135 -> 144 -> 117 -> 4 -> 115 -> 5 -> 39 -> 82 -> 2 -> 176 -> 123 -> 52 -> 59 -> 43 ->

In [35]:
filename = 'data/tsp_574_1'
OR_tsp_solver(filename)

Objective: 38328 miles
Route for vehicle 0:
 0 -> 4 -> 1 -> 2 -> 3 -> 455 -> 456 -> 457 -> 458 -> 459 -> 460 -> 463 -> 462 -> 461 -> 500 -> 499 -> 452 -> 453 -> 440 -> 441 -> 450 -> 449 -> 448 -> 447 -> 446 -> 442 -> 445 -> 444 -> 443 -> 439 -> 454 -> 438 -> 437 -> 436 -> 435 -> 434 -> 433 -> 432 -> 430 -> 431 -> 429 -> 428 -> 427 -> 426 -> 421 -> 420 -> 416 -> 415 -> 414 -> 413 -> 352 -> 351 -> 350 -> 349 -> 348 -> 347 -> 362 -> 363 -> 361 -> 360 -> 359 -> 358 -> 357 -> 354 -> 353 -> 355 -> 356 -> 412 -> 411 -> 410 -> 418 -> 417 -> 419 -> 422 -> 423 -> 424 -> 425 -> 382 -> 386 -> 387 -> 385 -> 383 -> 384 -> 409 -> 405 -> 404 -> 403 -> 402 -> 401 -> 400 -> 399 -> 406 -> 408 -> 407 -> 397 -> 398 -> 370 -> 369 -> 368 -> 367 -> 366 -> 329 -> 328 -> 327 -> 326 -> 325 -> 324 -> 323 -> 322 -> 373 -> 372 -> 371 -> 396 -> 395 -> 394 -> 393 -> 392 -> 391 -> 388 -> 389 -> 390 -> 381 -> 380 -> 379 -> 378 -> 377 -> 376 -> 375 -> 374 -> 315 -> 314 -> 316 -> 313 -> 312 -> 451 -> 311 -> 310 -> 309 ->

In [38]:
filename = 'data/tsp_1889_1'
OR_tsp_solver(filename)

Objective: 332021 miles
Route for vehicle 0:
 0 -> 3 -> 1866 -> 881 -> 1732 -> 1783 -> 1834 -> 1794 -> 548 -> 546 -> 385 -> 1253 -> 1252 -> 1021 -> 181 -> 255 -> 215 -> 480 -> 751 -> 1701 -> 1729 -> 923 -> 258 -> 437 -> 448 -> 502 -> 308 -> 661 -> 444 -> 384 -> 1416 -> 1616 -> 1617 -> 19 -> 372 -> 547 -> 1599 -> 541 -> 1046 -> 318 -> 386 -> 419 -> 966 -> 965 -> 964 -> 963 -> 162 -> 967 -> 407 -> 251 -> 1511 -> 1644 -> 1047 -> 1645 -> 1646 -> 1600 -> 316 -> 1589 -> 1647 -> 1648 -> 1656 -> 1657 -> 1501 -> 1658 -> 300 -> 1605 -> 706 -> 90 -> 20 -> 1651 -> 1726 -> 1590 -> 705 -> 1601 -> 783 -> 721 -> 1877 -> 1777 -> 1831 -> 1048 -> 968 -> 1260 -> 1259 -> 704 -> 1536 -> 782 -> 1233 -> 1830 -> 89 -> 1697 -> 1620 -> 1635 -> 45 -> 61 -> 719 -> 720 -> 1122 -> 1815 -> 252 -> 254 -> 1654 -> 476 -> 1261 -> 969 -> 196 -> 970 -> 214 -> 253 -> 531 -> 273 -> 127 -> 317 -> 315 -> 420 -> 520 -> 479 -> 408 -> 387 -> 148 -> 344 -> 471 -> 1602 -> 836 -> 1591 -> 373 -> 319 -> 323 -> 794 -> 568 -> 472 -> 271